# House Prices Advanced Regression - Neural Networks

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats.distributions import reciprocal

In [2]:
df_train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

In [3]:
X = df_train.drop(['SalePrice'], axis=1)
y = df_train['SalePrice']

# 1- Basic Preprocessing

Basic preprocessing transformer to perform feature_engineer and so on. Reduce uninportant variables.

In [4]:
num_cols = X.columns[X.dtypes != 'object']

In [5]:
num_cols

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

## 1.1 Numeric Variables

* **GrLivArea = 1stFlrSF + 2ndFlrSF + LowQualFinSF, one variable is redundant. Drop 2ndFlrSF. Drop LowQualFinSF, because it has few non zero values**

In [6]:
np.mean(X['GrLivArea'] == (X['1stFlrSF'] + X['2ndFlrSF'] + X['LowQualFinSF']))

1.0

In [7]:
np.mean(X['LowQualFinSF'] > 0)

0.01780821917808219

* **TotalBsmtSF = BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF. Redundancy, drop BsmtFinSF2**

In [8]:
np.mean(X['TotalBsmtSF'] == (X['BsmtFinSF1']+X['BsmtFinSF2']+X['BsmtUnfSF']))

1.0

* **BsmtHalfBath has almost no variance. Most of it is zero. Can be removed**

In [9]:
np.mean(X['BsmtHalfBath'] > 0)

0.056164383561643834

* **MiscVal should be removed, its number represents different things. To hard to deal with at this moment**

In [10]:
(X['MiscVal'] > 0).describe()

count      1460
unique        2
top       False
freq       1408
Name: MiscVal, dtype: object

* **Lot Frontage has missing values, which probably mean some houses have no Lot Frontage, input with  zeros**

In [11]:
sum(pd.isna(X['LotFrontage']))

259

* **GarageYearBlt has missing values (When there is no garage). Input with zero**

In [12]:
sum(pd.isna(X['GarageYrBlt'])) 

81

In [13]:
np.mean((X['GarageYrBlt'] == X['YearBuilt']) | (X['GarageYrBlt'] == X['YearRemodAdd']))

0.7828767123287671

* **MasVnrArea has missing values. Input with zero**

* **MoSold doesn't add much value to the model, would only add noise. The month a house is sold doesn't, have much impact on its price. Drop**

* **Most values of Pool Area are 0 (Means the house doesn't have a pool). Drop**

In [14]:
np.mean(X['PoolArea'] > 0), np.sum(X['PoolArea'] > 0)

(0.004794520547945206, 7)

In [15]:
class Preprocessing_Numeric(BaseEstimator, TransformerMixin):
    def __init__(self,age=True, grl=False):
        self.age = age
        self.grl = grl
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        # Dropping index
        Xt = Xt.drop(['Id'], axis=1)
        
        # Since GrLivArea = 1stFlrSF + 2ndFlrSF, one variable is redundant. Drop 2ndFlrSF.Drop LowQualFinSF, 
        #because it has few non zero values
        Xt = Xt.drop(['2ndFlrSF', 'LowQualFinSF'], axis=1)
        
        # Since TotalBsmtSF = BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF. Redundancy, drop BsmtFinSF2
        Xt = Xt.drop(['BsmtFinSF2'], axis=1)
        
        # BsmtHalfBath has almost no variance. Most of it is zero. Can be removed
        Xt = Xt.drop(['BsmtHalfBath'], axis=1)
        
        # MiscVal should be removed, its number represents different things. To hard to deal with at this moment
        Xt = Xt.drop(['MiscVal'], axis=1)
        
        # Lot Frontage has missing values, which probably mean some houses have no Lot Frontage, input with  zeros
        Xt['LotFrontage'] = Xt['LotFrontage'].fillna(0)
        
        # GarageYearBlt has missing values (When there is no garage). Input with zero
        Xt['GarageYrBlt'] = Xt['GarageYrBlt'].fillna(0)
        
        # MasVnrArea has missing values. Input with zero
        Xt['MasVnrArea'] = Xt['MasVnrArea'].fillna(0)
        
        # MoSold doesn't add much value to the model, would only add noise. 
        #The month a house is sold doesn't, have much impact on its price. Drop
        
        Xt = Xt.drop(['MoSold'], axis=1)
        
        # Most values of Pool Area are 0 (Means the house doesn't have a pool). Drop
        Xt = Xt.drop(['PoolArea'], axis=1)
        
        ''' Feature engineering options '''
        if self.age:
            # Takes the mean of the house age and the remodel age at the time of sale
            Xt['Age'] = (2*Xt['YrSold'] - Xt['YearBuilt'] - Xt['YearRemodAdd'])/2
            Xt = Xt.drop(['YrSold', 'YearBuilt', 'YearRemodAdd'], axis=1)
        if self.grl:
            # Multicollinearity between GrLivArea and 1stFlrSF --> Keep GrLivArea and 1stFlrSF/GrLivArea
            Xt['1stFlr_ratio'] = Xt['1stFlrSF']/Xt['GrLivArea']
            Xt = Xt.drop(['1stFlrSF'], axis=1)            
        return Xt

## 1.2 Ordinal Variables

## 1.3 Nominal Variables

# 2 - Full preprocessing Pipeline

Includes, Feature Engineering, Imputation, and Scaling

In [180]:
pipe_preprocessing = Pipeline([
    ('Preprocessing_Numeric', Preprocessing_Numeric()),
    #('Preprocessing_Ordinal', Preprocessing_Ordinal()),
    #('Preprocessing_Nominal', Preprocessing_Nominal())
    ('Imputation', SimpleImputer(strategy='constant', fill_value=0)),
    ('Scaling', StandardScaler())
])

# 3 - Creating a generic Sequential Neural Network Regresor with Keras

### 3.1 Creating the function to generate an RNA with given parameters

In [17]:
def Sequential_RNA_Regressor(n_hidden=2, n_neurons_hidden=30, 
                             activation_hidden='relu', weights_method='he_normal',
                            loss='mse',
                            lr=1e-3):
    # n_hidden = Number of Hidden Layers (Integer)
    # n_neurons_hidden = Number of neurons for each layer (Integer (Broadcast to all layers) or List (Each entry for each layer))
    # activation_hidden = Activation function for the hidden layers
    # weights_method = Method used to initialize weights (Default: he_normal | Good for RElU and variants, 
    # for SElU must be lecun_normal)
    
    
    # loss = Loss function for to be minimized
    # lr = Learning rate for the optimizer (In this case the keras.optimizers.SGD (Stochastic Gradient Descent))
    
    # If n_neurons_hidden is an integer, broadcast
    if isinstance(n_neurons_hidden, int):
        n_neurons_hidden = [n_neurons_hidden for i in range(n_hidden)]
    
    model = keras.models.Sequential()

    ## In this approach Keras will have to guess the input shape at the start of training. Because the input_shape depends
    # on the Preprocessing Pipeline hyperparameters chosen
    
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons_hidden[i], activation=activation_hidden, kernel_initializer=weights_method,
                 name='Hidden_{}'.format(i+1)))
    

    # Output layer. Regressor so just one neuron with no activation function
    model.add(keras.layers.Dense(1,name='Output_Layer'))
    
    # Compiling the model
    model.compile(loss=loss,
                 optimizer=keras.optimizers.SGD(learning_rate=lr))
    return model

### 3.2 Wrapping the keras model as an Sckit-Learn Regressor to make it compatible for hyperparameter tuning

In [18]:
rna_sk_regressor = keras.wrappers.scikit_learn.KerasRegressor(Sequential_RNA_Regressor)

C:\Users\vmrod\AppData\Local\Temp/ipykernel_14420/3889148913.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  rna_sk_regressor = keras.wrappers.scikit_learn.KerasRegressor(Sequential_RNA_Regressor)


# 4 - Searching for good hyperparameters

First of all, use EarlyStopping to prevent overfitting. For that, a validation set is needed, it'll be set at 15 % of the original training data.

In [183]:
# Obs: For test purposes only, X_train will have just numerical parameters, erase that later
X_train = X[num_cols].drop(['MSSubClass','OverallQual', 'OverallCond'], axis=1).copy()
X_train = pipe_preprocessing.fit_transform(X_train)

## 4.1 First search

A broad search to get a first impression where the look for the optimal hyperparameters.

In [168]:
param_dist_first_search = [
    # Option 1 | 2 layers
    {'n_hidden':[2],
     'n_neurons_hidden':[30, 50, 100, [150, 100]],
     'activation_hidden': ['selu'],
     'weights_method':['lecun_normal'],
     
     # Learning rate is going to be drawn from a loguniform random variable 
     # (Because since good candidates are not known beforehand, all orders of magnitude should have the same weight)

     #'lr':reciprocal(1e-3, 1)
    },
    
    # Option 2 | 3 layers
    {'n_hidden':[3],
     'n_neurons_hidden':[30, 50, [150, 100, 50]],
     'activation_hidden': ['selu'],
     'weights_method':['lecun_normal'],
     
    # Learning rate is going to be drawn from a loguniform random variable 
    # (Because since I don't know good candidates beforehand, all orders of magnitude should have the same weight)
      #'lr':reciprocal(1e-3, 1)
    }

]

In [169]:
# Obs: Predict logarithm of response (Magnitude varies a lot)
grid_first_search = RandomizedSearchCV(rna_sk_regressor, param_distributions=param_dist_first_search, 
                                       cv=6, n_iter=40, scoring='neg_mean_squared_error', verbose=4)

In [170]:
# Early stopping callback and model fit
'''keras.backend.clear_session()
early_cb = keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)
grid_first_search.fit(X_train, np.log(y), validation_split=0.15,
                     callbacks=[early_cb],
                     epochs=200)'''

Fitting 6 folds for each of 7 candidates, totalling 42 fits
Epoch 1/200


C:\Users\vmrod\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 7 is smaller than n_iter=40. Running 7 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


33/33 [==============================] - 0s 6ms/step - loss: 95.2340 - val_loss: 31.5951
Epoch 2/200
33/33 [==============================] - 0s 2ms/step - loss: 9.7044 - val_loss: 2.9226
Epoch 3/200
33/33 [==============================] - 0s 3ms/step - loss: 1.7706 - val_loss: 1.5592
Epoch 4/200
33/33 [==============================] - 0s 3ms/step - loss: 1.0978 - val_loss: 1.1214
Epoch 5/200
33/33 [==============================] - 0s 2ms/step - loss: 0.8419 - val_loss: 0.8923
Epoch 6/200
33/33 [==============================] - 0s 2ms/step - loss: 0.6974 - val_loss: 0.7959
Epoch 7/200
33/33 [==============================] - 0s 2ms/step - loss: 0.5940 - val_loss: 0.6720
Epoch 8/200
33/33 [==============================] - 0s 2ms/step - loss: 0.5274 - val_loss: 0.6391
Epoch 9/200
33/33 [==============================] - 0s 3ms/step - loss: 0.4689 - val_loss: 0.6031
Epoch 10/200
33/33 [==============================] - 0s 2ms/step - loss: 0.4316 - val_loss: 0.5324
Epoch 11/200
33/33 

33/33 [==============================] - 0s 2ms/step - loss: 0.2759 - val_loss: 0.2508
Epoch 14/200
33/33 [==============================] - 0s 2ms/step - loss: 0.2654 - val_loss: 0.2307
Epoch 15/200
33/33 [==============================] - 0s 2ms/step - loss: 0.2513 - val_loss: 0.3671
Epoch 16/200
33/33 [==============================] - 0s 2ms/step - loss: 0.2438 - val_loss: 0.2436
Epoch 17/200
33/33 [==============================] - 0s 2ms/step - loss: 0.2331 - val_loss: 0.2595
Epoch 18/200
33/33 [==============================] - 0s 2ms/step - loss: 0.2243 - val_loss: 0.2415
Epoch 19/200
33/33 [==============================] - 0s 2ms/step - loss: 0.2156 - val_loss: 0.2408
Epoch 20/200
33/33 [==============================] - 0s 2ms/step - loss: 0.2104 - val_loss: 0.2357
Epoch 21/200
33/33 [==============================] - 0s 2ms/step - loss: 0.2034 - val_loss: 0.2228
Epoch 22/200
33/33 [==============================] - 0s 2ms/step - loss: 0.1975 - val_loss: 0.2241
Epoch 23/200


33/33 [==============================] - 0s 2ms/step - loss: 0.0721 - val_loss: 0.1474
Epoch 96/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0719 - val_loss: 0.1464
Epoch 97/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0710 - val_loss: 0.1526
Epoch 98/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0701 - val_loss: 0.1523
Epoch 99/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0701 - val_loss: 0.1506
Epoch 100/200
33/33 [==============================] - 0s 3ms/step - loss: 0.0693 - val_loss: 0.1549
Epoch 101/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0695 - val_loss: 0.1486
Epoch 102/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0688 - val_loss: 0.1502
Epoch 103/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0684 - val_loss: 0.1468
Epoch 104/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0679 - val_loss: 0.1461
Epoch 10

...

Epoch 113/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0427 - val_loss: 0.1526
Epoch 114/200
33/33 [==============================] - 0s 3ms/step - loss: 0.0424 - val_loss: 0.1539
Epoch 115/200
33/33 [==============================] - 0s 3ms/step - loss: 0.0430 - val_loss: 0.1533
Epoch 116/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0425 - val_loss: 0.1514
Epoch 117/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0419 - val_loss: 0.1525
Epoch 118/200
33/33 [==============================] - 0s 3ms/step - loss: 0.0415 - val_loss: 0.1515
Epoch 119/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0409 - val_loss: 0.1517
Epoch 120/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0414 - val_loss: 0.1429
Epoch 121/200
33/33 [==============================] - 0s 3ms/step - loss: 0.0400 - val_loss: 0.1473
Epoch 122/200
33/33 [==============================] - 0s 3ms/step - loss: 0.0393 - val_los

33/33 [==============================] - 0s 3ms/step - loss: 0.0261 - val_loss: 0.1291
Epoch 195/200
33/33 [==============================] - 0s 3ms/step - loss: 0.0261 - val_loss: 0.1297
Epoch 196/200
33/33 [==============================] - 0s 3ms/step - loss: 0.0256 - val_loss: 0.1333
Epoch 197/200
33/33 [==============================] - 0s 3ms/step - loss: 0.0262 - val_loss: 0.1276
Epoch 198/200
33/33 [==============================] - 0s 2ms/step - loss: 0.0262 - val_loss: 0.1281
Epoch 199/200
33/33 [==============================] - 0s 3ms/step - loss: 0.0252 - val_loss: 0.1306
Epoch 200/200
8/8 [==============================] - 0s 2ms/step
[CV 6/6] END activation_hidden=selu, n_hidden=3, n_neurons_hidden=[150, 100, 50], weights_method=lecun_normal;, score=-0.204 total time=  19.4s
Epoch 1/200
39/39 [==============================] - 1s 5ms/step - loss: 65.4474 - val_loss: 3.2815
Epoch 2/200
39/39 [==============================] - 0s 2ms/step - loss: 1.2679 - val_loss: 0.6263


39/39 [==============================] - 0s 2ms/step - loss: 0.0576 - val_loss: 0.1077
Epoch 75/200
39/39 [==============================] - 0s 2ms/step - loss: 0.0569 - val_loss: 0.1075
Epoch 76/200
39/39 [==============================] - 0s 2ms/step - loss: 0.0563 - val_loss: 0.1071
Epoch 77/200
39/39 [==============================] - 0s 2ms/step - loss: 0.0560 - val_loss: 0.1065
Epoch 78/200
39/39 [==============================] - 0s 2ms/step - loss: 0.0554 - val_loss: 0.1060
Epoch 79/200
39/39 [==============================] - 0s 2ms/step - loss: 0.0551 - val_loss: 0.1057
Epoch 80/200
39/39 [==============================] - 0s 2ms/step - loss: 0.0545 - val_loss: 0.1054
Epoch 81/200
39/39 [==============================] - 0s 2ms/step - loss: 0.0541 - val_loss: 0.1048
Epoch 82/200
39/39 [==============================] - 0s 2ms/step - loss: 0.0536 - val_loss: 0.1041
Epoch 83/200
39/39 [==============================] - 0s 2ms/step - loss: 0.0533 - val_loss: 0.1039
Epoch 84/200


39/39 [==============================] - 0s 3ms/step - loss: 0.0368 - val_loss: 0.0862
Epoch 156/200
39/39 [==============================] - 0s 3ms/step - loss: 0.0367 - val_loss: 0.0863
Epoch 157/200
39/39 [==============================] - 0s 3ms/step - loss: 0.0366 - val_loss: 0.0861
Epoch 158/200
39/39 [==============================] - 0s 3ms/step - loss: 0.0364 - val_loss: 0.0858
Epoch 159/200
39/39 [==============================] - 0s 3ms/step - loss: 0.0364 - val_loss: 0.0858
Epoch 160/200
39/39 [==============================] - 0s 3ms/step - loss: 0.0363 - val_loss: 0.0856
Epoch 161/200
39/39 [==============================] - 0s 4ms/step - loss: 0.0361 - val_loss: 0.0856
Epoch 162/200
39/39 [==============================] - 0s 3ms/step - loss: 0.0360 - val_loss: 0.0855
Epoch 163/200
39/39 [==============================] - 0s 3ms/step - loss: 0.0359 - val_loss: 0.0855
Epoch 164/200
39/39 [==============================] - 0s 3ms/step - loss: 0.0357 - val_loss: 0.0850
Epoc

RandomizedSearchCV(cv=6,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000001DF0003F4C0>,
                   n_iter=40,
                   param_distributions=[{'activation_hidden': ['selu'],
                                         'n_hidden': [2],
                                         'n_neurons_hidden': [30, 50, 100,
                                                              [150, 100]],
                                         'weights_method': ['lecun_normal']},
                                        {'activation_hidden': ['selu'],
                                         'n_hidden': [3],
                                         'n_neurons_hidden': [30, 50,
                                                              [150, 100, 50]],
                                         'weights_method': ['lecun_normal']}],
                   scoring='neg_mean_squared_error', verbose=4)

In [174]:
grid_first_search.best_params_, grid_first_search.best_params_

({'weights_method': 'lecun_normal',
  'n_neurons_hidden': 30,
  'n_hidden': 3,
  'activation_hidden': 'selu'},
 {'weights_method': 'lecun_normal',
  'n_neurons_hidden': 30,
  'n_hidden': 3,
  'activation_hidden': 'selu'})

# 5 -  Model Evaluation and Predictions

## 5.1 Model Evaluation

## 5.2 Predictions

In [184]:
def write_test(X_test, processing_pipe, model, name):
    X_pr = processing_pipe.transform(X_test)
    pred = np.exp(model.predict(X_pr)) #Inverting the log transformation
    df_pred = pd.concat([X_test.Id, pd.Series(pred, name='SalePrice')], axis=1)
    df_pred.to_csv(name, index=False)